# Vector Search with Qdrant

In [1]:
import docs_08 as docs

github_data = docs.read_github_data('evidentlyai', 'docs')
parsed_data = docs.parse_data(github_data)
chunks = docs.chunk_documents(parsed_data)


Run Qdrant:

```bash
docker run -it --rm \
    -p 6333:6333 -p 6334:6334 \
    -v "qdrant_storage:/qdrant/storage:z" \
    qdrant/qdrant
```

Install the python module:

In [3]:
!uv add 'qdrant-client[fastembed]'

Resolved 172 packages in 1.11s                                       
⠙ Preparing packages... (0/15)                                                  ⠋ Preparing packages... (0/0)                                                   
⠙ Preparing packages... (0/15)-------------     0 B/60.15 KiB           
⠙ Preparing packages... (0/15)------------- 16.00 KiB/60.15 KiB         
⠙ Preparing packages... (0/15)------------- 32.00 KiB/60.15 KiB         
⠙ Preparing packages... (0/15)--------- 48.00 KiB/60.15 KiB         
⠙ Preparing packages... (0/15)--------- 60.15 KiB/60.15 KiB         
⠙ Preparing packages... (0/15)--------- 60.15 KiB/60.15 KiB         
⠙ Preparing packages... (0/15)--------- 60.15 KiB/60.15 KiB         
portalocker          ------------------------------     0 B/21.90 KiB
⠙ Preparing packages... (0/15)--------- 60.15 KiB/60.15 KiB         
⠙ Preparing packages... (0/15)-------------     0 B/21.90 KiB           
⠙ Preparing packages... (0/15)--------- 16.00 KiB/21.90 KiB   

Use it in Python:

In [4]:
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")

Create the collection:

In [5]:
dimensionality = 512
model_name = "jinaai/jina-embeddings-v2-small-en"
collection_name = "evidently-docs"


# if you want to re-insert the data, delete the collection first
# qd_client.delete_collection(collection_name=collection_name)


qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=dimensionality,
        distance=models.Distance.COSINE
    )
)

True

Ingest:

In [6]:
points = []

for i, doc in enumerate(chunks):
    text = doc.get('title', '') + ' ' + doc.get('description', '') + ' ' + doc['content']
    text = text.strip()
    vector = models.Document(text=text, model=model_name)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)


qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

## Retrieval

In [7]:
def vector_search(query, num_results=15):
    vector = models.Document(text=query, model=model_name)

    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=vector,
        limit=num_results,
        with_payload=True
    )

    results = []

    for point in query_points.points:
        results.append(point.payload)

    return results

In [8]:
search_result = vector_search('how do I use llm-as-a-judge for evals')
print(search_result)

[{'start': 20000, 'content': 'openai",\n            model = "gpt-4o-mini",\n            alias="Verbosity")\n    ])\n```\n\nRun the Report and view the summary results:\xa0\n\n```python\nreport = Report([\n    TextEvals()\n])\n\nmy_eval = report.run(eval_dataset, None)\nmy_eval\n```\n\n![](/images/examples/llm_judge_tutorial_verbosity-min.png)\n\nYou can also view the dataframe using `eval_dataset.as_dataframe()`\n\n<Info>\n  Don\'t fully agree with the results? Use these labels as a starting point, edit the decisions where you see fit - now you\'ve got your golden dataset\\! Next, iterate on your judge prompt. You can also try different evaluator LLMs to see which one does the job better. [How to change an LLM](/metrics/customize_llm_judge#change-the-evaluator-llm).\n</Info>\n\n## What\'s next?\n\nThe LLM judge itself is just one part of your overall evaluation framework. You can integrate this evaluator into different workflows, such as testing your LLM outputs after changing a prompt

## RAG

In [9]:
def search(query):
    results = vector_search(
        query=query,
        num_results=15
    )

    return results

In [10]:
import json

instructions = """
You're an assistant that helps with the documentation.
Answer the QUESTION based on the CONTEXT from the search engine of our documentation.

Use only the facts from the CONTEXT when answering the QUESTION.

When answering the question, provide the reference to the file with the source.
Use the filename field for that. The repo url is: https://github.com/evidentlyai/docs/
Include code examples when relevant. 
If the question is discussed in multiple documents, cite all of them.

Don't use markdown or any formatting in the output.
""".strip()

prompt_template = """
<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()


def build_prompt(question, search_results):
    context = json.dumps(search_results)

    prompt = prompt_template.format(
        question=question,
        context=context
    ).strip()
    
    return prompt

In [12]:
from openai import OpenAI

openai_client = OpenAI()

def llm(user_prompt, instructions=None, model="gpt-4o-mini"):
    messages = []

    if instructions:
        messages.append({
            "role": "system",
            "content": instructions
        })

    messages.append({
        "role": "user",
        "content": user_prompt
    })

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text

In [13]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    response = llm(prompt)
    return response

In [14]:
result = rag("How can I build an eval report with llm as a judge?")
print(result)

To create an evaluation (eval) report using an LLM as a judge, you'll want to follow these steps:

### 1. Environment Setup
1. **Install Required Libraries**:
   Make sure you have Python installed, and then run:
   ```bash
   pip install evidently
   ```

2. **Import Necessary Modules**:
   ```python
   import pandas as pd
   import os
   from evidently import Dataset, DataDefinition, Report
   from evidently.llm.templates import BinaryClassificationPromptTemplate
   ```

3. **Set Your OpenAI API Key**:
   ```python
   os.environ["OPENAI_API_KEY"] = "YOUR_KEY"
   ```

### 2. Create an Evaluation Dataset
1. **Create a Toy Dataset**:
   Construct a sample dataset with questions, target responses, new responses, and manual labels.
   ```python
   data = [["question1", "target response 1", "new response 1", "correct", "comment"],
           ["question2", "target response 2", "new response 2", "incorrect", "comment"]]
   columns = ["question", "target_response", "new_response", "label", "c